# Qwen 2.5 3B Counting Benchmark + Mediation Analysis

This notebook runs on Google Colab with T4 GPU:
1. Benchmarks Qwen 2.5 3B on the counting task
2. Performs activation patching mediation analysis to identify which layers causally mediate count information

## Setup: Install Dependencies

In [ ]:
!pip install -q transformers accelerate torch tqdm

## Clone Repository and Load Dataset

In [ ]:
!git clone https://github.com/Larsen-Daniel/llm-counting-analysis.git
%cd llm-counting-analysis

## Check GPU Availability

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. Model will run on CPU (very slow).")
    print("Make sure you've enabled GPU in Runtime > Change runtime type")

## Benchmark Code

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict
from tqdm import tqdm
import re

def load_dataset(file_path: str) -> List[Dict]:
    """Load the dataset from JSONL file."""
    examples = []
    with open(file_path, 'r') as f:
        for line in f:
            examples.append(json.loads(line))
    return examples

def extract_answer(text: str) -> int | None:
    """
    Extract numerical answer from model output.
    Try multiple patterns in order of preference:
    1. Last number in parentheses: (N)
    2. First number on first line
    """
    # First try to find number in parentheses
    matches = re.findall(r'\((\d+)\)', text)
    if matches:
        return int(matches[-1])  # Return the last match

    # Fall back to first number in the output
    first_line = text.strip().split('\n')[0]
    number_match = re.search(r'\d+', first_line)
    if number_match:
        return int(number_match.group(0))

    return None  # No number found

def benchmark_qwen(
    model_name: str,
    dataset: List[Dict],
    max_examples: int = None,
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
) -> Dict:
    """
    Benchmark Qwen model on the counting task.

    Args:
        model_name: HuggingFace model name
        dataset: List of examples
        max_examples: Maximum examples to evaluate
        device: Device to run on

    Returns:
        Dictionary with results
    """
    print(f"\n{'='*80}")
    print(f"Benchmarking: {model_name}")
    print(f"Device: {device}")
    print(f"{'='*80}\n")

    # Load model and tokenizer
    print("Loading model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None,
        low_cpu_mem_usage=True
    )

    if device == "cpu":
        model = model.to(device)

    model.eval()

    # Prepare dataset
    if max_examples:
        dataset = dataset[:max_examples]

    results = {
        "model_id": model_name,
        "total_examples": len(dataset),
        "correct": 0,
        "incorrect": 0,
        "parse_errors": 0,
        "numerical_errors": 0,
        "api_errors": 0,
        "predictions": []
    }

    print(f"Evaluating on {len(dataset)} examples...")

    for example in tqdm(dataset):
        prompt = example["prompt"]
        true_answer = example["answer"]

        # Generate
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=10,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        generated_text = tokenizer.decode(
            outputs[0][inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        )

        # Extract answer
        predicted_answer = extract_answer(generated_text)

        # Check correctness
        if predicted_answer is None:
            results["parse_errors"] += 1
            is_correct = False
        else:
            is_correct = predicted_answer == true_answer
            if is_correct:
                results["correct"] += 1
            else:
                results["numerical_errors"] += 1
                results["incorrect"] += 1

        # Store prediction
        results["predictions"].append({
            "prompt": prompt,
            "true_answer": true_answer,
            "predicted_answer": predicted_answer,
            "generated_text": generated_text,
            "correct": is_correct
        })

    # Calculate metrics
    results["accuracy"] = results["correct"] / results["total_examples"]
    results["parse_error_rate"] = results["parse_errors"] / results["total_examples"]
    results["numerical_error_rate"] = results["numerical_errors"] / results["total_examples"]
    results["api_error_rate"] = results["api_errors"] / results["total_examples"]

    print(f"\nResults for {model_name}:")
    print(f"  Accuracy: {results['accuracy']:.2%} ({results['correct']}/{results['total_examples']})")
    print(f"  Parse Errors: {results['parse_error_rate']:.2%}")
    print(f"  Numerical Errors: {results['numerical_error_rate']:.2%}")

    return results

## Run Benchmark

In [ ]:
# Load dataset
dataset = load_dataset('data/counting_dataset.jsonl')
print(f"Loaded {len(dataset)} examples")

# Auto-detect device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Run benchmark on 1000 examples
results = benchmark_qwen(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    dataset=dataset,
    max_examples=1000,
    device=device
)

# Save results
with open('results/qwen_benchmark_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\nResults saved to results/qwen_benchmark_results.json")

## Download Results

In [ ]:
from google.colab import files
files.download('results/qwen_benchmark_results.json')

## Load Model for Mediation Analysis

Run this cell to load the model (if you haven't run the benchmark above, or if you want to reload the model)

In [ ]:
# Load model and tokenizer for mediation analysis
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model_name = "Qwen/Qwen2.5-3B-Instruct"
print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    low_cpu_mem_usage=True
)

if device == "cpu":
    model = model.to(device)

model.eval()
print(f"Model loaded successfully!")
print(f"Model device: {next(model.parameters()).device}")

In [ ]:
# Pull latest changes and reload
!git pull origin main

# Import after adding scripts to path
import sys
sys.path.append('scripts')

import importlib
import mediation_utils
importlib.reload(mediation_utils)

from mediation_utils import run_mediation_analysis, extract_answer
import json
import random

# Auto-detect device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load dataset
print("\nLoading dataset...")
dataset = []
with open('data/counting_dataset.jsonl', 'r') as f:
    for line in f:
        dataset.append(json.loads(line))
print(f"Loaded {len(dataset)} examples")

# Generate simple synthetic pairs - SAME AS BENCHMARK TEST
def generate_simple_pairs(n_pairs=500):
    """Generate pairs matching dataset format: 8-10 word lists with 2-3 matching items."""
    CATEGORIES = {
        "fruit": ["apple", "banana", "cherry", "grape", "orange", "mango", "kiwi", "peach", "pear", "plum",
                  "strawberry", "blueberry", "watermelon", "pineapple", "lemon"],
        "animal": ["dog", "cat", "bird", "fish", "horse", "cow", "pig", "sheep", "goat", "chicken",
                   "rabbit", "mouse", "elephant", "lion", "tiger"],
        "vehicle": ["car", "bus", "truck", "bike", "train", "plane", "boat", "ship", "motorcycle", "scooter",
                    "van", "taxi", "subway", "helicopter", "tram"],
        "color": ["red", "blue", "green", "yellow", "orange", "purple", "pink", "brown", "black", "white",
                  "gray", "violet", "indigo", "turquoise", "crimson"],
        "tool": ["hammer", "screwdriver", "wrench", "pliers", "saw", "drill", "chisel", "axe", "knife", "scissors",
                 "ruler", "tape", "level", "clamp", "file"],
        "furniture": ["chair", "table", "sofa", "bed", "desk", "cabinet", "shelf", "dresser", "bench", "stool",
                      "wardrobe", "bookcase", "ottoman", "nightstand", "couch"],
        "clothing": ["shirt", "pants", "dress", "skirt", "jacket", "coat", "sweater", "hat", "shoes", "socks",
                     "scarf", "gloves", "belt", "tie", "jeans"],
        "food": ["pizza", "burger", "pasta", "rice", "bread", "cheese", "salad", "soup", "sandwich", "taco",
                 "noodles", "steak", "chicken", "fish", "potato"],
        "sport": ["soccer", "basketball", "tennis", "baseball", "football", "hockey", "volleyball", "golf", "cricket", "rugby",
                  "swimming", "boxing", "skiing", "skating", "running"],
        "instrument": ["guitar", "piano", "drums", "violin", "flute", "trumpet", "saxophone", "cello", "clarinet", "harp",
                       "trombone", "banjo", "accordion", "oboe", "tuba"]
    }

    NOISE_WORDS = ["bowl", "window", "door", "cloud", "mountain", "river", "tree", "rock", "sand", "metal",
                   "plastic", "glass", "paper", "wood", "stone", "gold", "silver", "copper", "iron", "steel",
                   "number", "letter", "word", "sentence", "paragraph", "page", "book", "magazine", "newspaper", "document"]

    pairs = []
    categories = list(CATEGORIES.keys())

    for i in range(n_pairs):
        category = categories[i % len(categories)]
        cat_words = CATEGORIES[category]

        list_length = random.randint(8, 10)
        num_matching = random.randint(2, 3)
        matching = random.sample(cat_words, num_matching)

        non_matching_pool = NOISE_WORDS.copy()
        for cat, items in CATEGORIES.items():
            if cat != category:
                non_matching_pool.extend(items)

        non_matching = random.sample(non_matching_pool, list_length - num_matching)

        base_list = matching + non_matching
        random.shuffle(base_list)

        low_list = base_list.copy()
        if low_list[0] in cat_words:
            available_noise = [w for w in non_matching_pool if w not in low_list and w not in cat_words]
            if not available_noise:
                continue
            low_list[0] = random.choice(available_noise)
        low_answer = sum(1 for w in low_list if w in cat_words)

        high_list = base_list.copy()
        if high_list[0] not in cat_words:
            available_cat = [w for w in cat_words if w not in high_list]
            if not available_cat:
                continue
            high_list[0] = random.choice(available_cat)
        high_answer = sum(1 for w in high_list if w in cat_words)

        if high_answer - low_answer != 1:
            continue

        prompt_template = """Count how many words in the list below match the given type.

Type: {category}
List: {word_list}

YOU MUST respond with ONLY a number in parentheses, like this: (5)
Do NOT include any other text, explanations, or words.
Just output the number in parentheses and nothing else.

Answer: """

        pair_low = {
            'prompt': prompt_template.format(category=category, word_list=' '.join(low_list)),
            'word_list': low_list,
            'category': category,
            'answer': low_answer
        }

        pair_high = {
            'prompt': prompt_template.format(category=category, word_list=' '.join(high_list)),
            'word_list': high_list,
            'category': category,
            'answer': high_answer
        }

        pairs.append((pair_low, pair_high))

    return pairs

# Generate 2000 candidate pairs (will be filtered to 100 perfect ones)
print("\nGenerating synthetic minimal pairs...")
pairs = generate_simple_pairs(n_pairs=2000)
print(f"Generated {len(pairs)} valid pairs")

print(f"\nRunning mediation analysis (will filter to 100 perfect examples)...")
mediation_results = run_mediation_analysis(model, tokenizer, pairs, device=device)

# Save results
with open('results/mediation_results_qwen3b.json', 'w') as f:
    json.dump(mediation_results, f, indent=2)

# Display results
print("\n" + "="*80)
print("TOP 5 LAYERS BY MEAN EFFECT")
print("="*80)
effects = [(int(i), data['mean_effect']) for i, data in mediation_results['layer_effects'].items()]
effects.sort(key=lambda x: x[1], reverse=True)
for layer, effect in effects[:5]:
    print(f"  Layer {layer}: {effect:.3f}")

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"Baseline accuracy: {mediation_results['baseline_accuracy']:.1%}")
print(f"Number of pairs tested: {mediation_results['n_pairs']}")

from google.colab import files
files.download('results/mediation_results_qwen3b.json')